In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [66]:
def save_in_file(items, file_name):
    file = open(file_name, "w")
    for item in items:
        file.write(item + "<myCustomTag>")
    file.close()

def get_from_files(file_name):
    file = open(file_name, "r")
    items = file.read().split("<myCustomTag>")
    file.close()

    return items[:-1]

In [3]:
def get_parsed_page(url):
    page = requests.get(url)
    return BeautifulSoup(page.text, 'html.parser')

def get_top_10_movies_ids():
    parsed_page = get_parsed_page("https://www.allocine.fr/film/meilleurs/")

    films_links = parsed_page.find_all('a', class_='meta-title-link')
    films_ids = []
    for film_link in films_links:
        films_ids.append(film_link['href'].split('cfilm=')[1].split('.')[0])
    
    return films_ids

def get_all_comments_page_links(parsed_page, URL):
    pagination = parsed_page.find('nav', class_='pagination')
    if pagination:
        last_page_element = pagination.find_all(['a', 'span'], class_='item')[-1]
        # last_page_link = last_page_element['href'] if last_page_element.name == 'a' else URL
        last_page_number = int(last_page_element.get_text())
        pages_urls = [f"{URL}?page={page}" for page in range(1, last_page_number + 1)]
    else:
        pages_urls = [URL]

    return pages_urls

def get_comments(parsed_page):
    comments = []
    for review in parsed_page.find_all('div', class_='content-txt review-card-content'):
        for spoiler in review.find_all('span', class_='spoiler-header'):
            spoiler.extract()

        comments.append(review.get_text(strip=True))
    
    return comments

def get_ratings(parsed_page):
        ratings = []
        # Récupérer les notes
        for review in parsed_page.find_all("div", class_="review-card-review-holder"):
            ratings.append(review.find("span", class_="stareval-note").get_text())
        
        return ratings

def scrap_comments(film_id):
    URL = f"https://www.allocine.fr/film/fichefilm-{film_id}/critiques/spectateurs/"
    parsed_movie_page = get_parsed_page(URL)

    pages_urls = get_all_comments_page_links(parsed_movie_page, URL)

    movie_comments = []
    movie_ratings = []
    page_number = 1
    for page_url in pages_urls:
        print("page_number => ", page_number)
        page_number+=1
        
        time.sleep(1)  # Attendre 1 seconde avant de faire la prochaine requête
        parsed_page = get_parsed_page(page_url)

        comments = get_comments(parsed_page)
        for comment in comments:
            movie_comments.append(comment)
        
        ratings = get_ratings(parsed_page)
        for rating in ratings:
            movie_ratings.append(rating)

        if(len(ratings) != len(comments)):
            print("=========================")

    return movie_comments, movie_ratings





In [4]:
film_ids = get_top_10_movies_ids()

all_comments = []
all_ratings = []
for film_id in film_ids:
    movie_comments, movie_ratings = scrap_comments(film_id)

    for movie_comment in movie_comments:
        all_comments.append(movie_comment)
    for movie_rating in movie_ratings:
        all_ratings.append(movie_rating)
    
    print("must break")
    break # TODO: Scrap all
save_in_file(all_comments, "scrapped_1_movie_comments.txt")
save_in_file(all_ratings, "scrapped_1_movie_ratings.txt")


page_number =>  1
page_number =>  2
page_number =>  3
page_number =>  4
page_number =>  5
page_number =>  6
page_number =>  7
page_number =>  8
page_number =>  9
page_number =>  10
page_number =>  11
page_number =>  12
page_number =>  13
page_number =>  14
page_number =>  15
page_number =>  16
page_number =>  17
page_number =>  18
page_number =>  19
page_number =>  20
page_number =>  21
page_number =>  22
page_number =>  23
page_number =>  24
page_number =>  25
page_number =>  26
page_number =>  27
page_number =>  28
page_number =>  29
page_number =>  30
page_number =>  31
page_number =>  32
page_number =>  33
page_number =>  34
page_number =>  35
page_number =>  36
page_number =>  37
page_number =>  38
page_number =>  39
page_number =>  40
page_number =>  41
page_number =>  42
page_number =>  43
page_number =>  44
page_number =>  45
page_number =>  46
page_number =>  47
page_number =>  48
page_number =>  49
page_number =>  50
page_number =>  51
page_number =>  52
page_number =>  53
pa

In [69]:
binaryRating = []
for rating in get_from_files("scrapped_1_movie_ratings.txt"):
    if rating.split(",")[0] in ["3","4","5"]:
        binaryRating.append(1)
    else :
        binaryRating.append(0)

len(binaryRating)


2364